In [169]:
import numpy as np
import pandas as pd

# read file into panda df
def pandas_reader(filename):
    df = pd.read_csv(filename)
    df = df.set_index('id')
    return df
def convert_numpy(train):
    # x be all the words
    train_x = train[list(train.keys())[:-1]]
    # let y be the label of trainging set
    train_y = train['label']
    np_train_x = train_x.to_numpy() 
    np_train_y = train_y.to_numpy()
    return np_train_x, np_train_y

    return np_train_x, np_train_y
def export_csv(test_noans, out_label):
    # put labels into test with noans first
    test_noans["label"] = out_label

    # get id and label col
    test_answer = test_noans["label"]

    # export to csv
    test_answer.to_csv("test_answer.csv")


In [209]:
train = pandas_reader("train.csv")
x, y = convert_numpy(train)
test = pandas_reader("test_noans.csv")
test_x = test.to_numpy() 

In [115]:
print(x, len(x))
print(y, len(y))

[[1 0 1 ... 0 0 0]
 [1 0 2 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 3 0 ... 0 0 0]
 [1 5 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]] 3000
[0 0 0 ... 0 0 1] 3000


In [14]:
"""

Initialize the weight vector w and the learning rate α; 
while objective function not converged do:
    Randomly shuffle the order of the examples in the training set;
    for each example in the training set do:
    w = w + α((y − p)x − 2λw );
    end 
end


"""

'\n\nInitialize the weight vector w and the learning rate α; \nwhile objective function not converged do:\n    Randomly shuffle the order of the examples in the training set;\n    for each example in the training set do:\n    w = w + α((y − p)w − 2λw );\n    end \nend\n\n\n'

In [224]:
# this function will calculate p using dot product
# p = 1 / (1 + e^ -(x*w))
def getp(x, w):
    z = np.dot(x, w)
    p = 1 / (1 + np.exp(-z))
    return p
# returns λ||w||2
def getNorm(lamb, w):
    norm = 0
    for el in w:
        norm += el**2
    norm = np.sqrt(norm)
    norm = lamb* norm
    return norm

# w is the initilized weight vector
# a is the learning rate, lamb is lambda.
# we should output the converged weight vector
def SGD(x, y, w, a, lamb):
    
    # calculte LCL based on the current weight vector
    # LCL = sum of (yi*log p + (1-yi)* log(1-p))
    # for every i, yi is the label, and p should be different
    LCL = 0
    new = 0
    prev = 0
    for i in range(len(x)):
        p = getp(x[i], w) # the p for this data
        LCL += (y[i]*np.log(p) + (1-y[i]) * np.log(1-p)) # sum up the LCL
    
    norm = getNorm(lamb, w)
    new = LCL - norm
    
    # stop the while loop when further iterations won't change the weight anymore
    # checking if the change in LCL − λ||w||2 is less than a small value 1.0 × 10−3 or 1.0 × 10−4
    # ||w||2 = sqrt(w1^2 + w2^2 + w3^2 + ... wn^2)
    #  while "objective function do not converge":
    epoch = 0

    while np.abs(new - prev) > 10 ** -3 :
        print(np.abs(new - prev), p, y[-1])
        epoch += 1
        
        print("epoch:", epoch)
#         if epoch > 150:
#             break
            
        # shuffle the training data
        # Get a random permutation of indices
        indices = np.random.permutation(len(x))

        # Shuffle x and y using the same permutation
        x = x[indices]
        y = y[indices]
        
        for i in range(len(x)):
            p = getp(x[i], w) # current p value
            # w = w + α((y − p)x − 2λw );
            res = [element * 2 * lamb for element in w] # 2 *lamb * w

            w = w + a*((y[i] - p)*x[i] - res)
        
        prev = new # record the previous LCL - norm
        LCL = 0 # recalculate new LCL here
        for i in range(len(x)):
            p = getp(x[i], w) # the p for this data
            LCL += (y[i]*np.log(p) + (1-y[i]) * np.log(1-p)) # sum up the LCL
        norm = getNorm(lamb, w)
        new = LCL - norm
        # dacay learning rate
        a = 0.975 * a

    # after the while loop ends, we return the optimized weight

    return w

In [179]:
# for 10 fold CV we need to train the model on 10 different data set, each one with a 10% held out training data
# after training, we use the held out data to test. And we will take the average of all the accuracy.
# w is the initilized weight vector
# a is the learning rate, lamb is lambda.
# we should output the converged weight vector
def SGD_test(x, y, w, a, lamb, held_x, held_y):
    
    # calculte LCL based on the current weight vector
    # LCL = sum of (yi*log p + (1-yi)* log(1-p))
    # for every i, yi is the label, and p should be different
    LCL = 0
    new = 0
    prev = 0
    for i in range(len(x)):
        p = getp(x[i], w) # the p for this data
        LCL += (y[i]*np.log(p) + (1-y[i]) * np.log(1-p)) # sum up the LCL
    
    norm = getNorm(lamb, w)
    new = LCL - norm
    
    # stop the while loop when further iterations won't change the weight anymore
    # checking if the change in LCL − λ||w||2 is less than a small value 1.0 × 10−3 or 1.0 × 10−4
    # ||w||2 = sqrt(w1^2 + w2^2 + w3^2 + ... wn^2)
    #  while "objective function do not converge":
    epoch = 0
    max_acc = 0
    max_epoch = 0
    while np.abs(new - prev) > 10 ** -3 :
        print(np.abs(new - prev), p, y[-1])
        epoch += 1
        
        print("epoch:", epoch)
#         if epoch > 100:
#             break
            
        # shuffle the training data
        # Get a random permutation of indices
        indices = np.random.permutation(len(x))

        # Shuffle x and y using the same permutation
        x = x[indices]
        y = y[indices]
        
        for i in range(len(x)):
            p = getp(x[i], w) # current p value
            # w = w + α((y − p)x − 2λw );
            res = [element * 2 * lamb for element in w] # 2 *lamb * w

            w = w + a*((y[i] - p)*x[i] - res)
        
        prev = new # record the previous LCL - norm
        LCL = 0 # recalculate new LCL here
        for i in range(len(x)):
            p = getp(x[i], w) # the p for this data
            LCL += (y[i]*np.log(p) + (1-y[i]) * np.log(1-p)) # sum up the LCL
        norm = getNorm(lamb, w)
        new = LCL - norm
        # dacay learning rate
        a = 0.98 * a
        
        # test the accuracy rate
        correct = 0
        total = len(held_x)
        # loop through these 
        for j in range(len(held_x)):
            pj = getp(held_x[j], w)
            p1 = pj # probability of label = 1
            p0 = 1 - pj # probability of label = 0
            if p1 > p0 :
                # we predict 1
                if y[j] == 1:
                    correct += 1 
            else:
                if y[j] == 0:
                    correct += 1
        accuracy = correct / total
        print("accuracy:", accuracy)
        if accuracy > max_acc:
            max_acc = accuracy
            max_epoch = epoch
    # after the while loop ends, we return the optimized weight
    
    print("max accuracy:", max_acc)
    print("epoch peaked at:", max_epoch)
    return w

3

In [182]:
A = [1,2,3,4]
print(A[-1])

4


In [117]:
def predict(w, test_x):
    # compute the labels using the weight vectors we have
    output = []
    for j in range(len(test_x)):
        p = getp(test_x[j], w)
        p1 = p # probability of label = 1
        p0 = 1 - p # probability of label = 0
        if p1 > p0 :
            output.append(1)
            
        else:
            output.append(0)
        
    # print(output)

    export_csv(test, output)

In [225]:
# the variable used for running the model
w = [0] * len(x[0])
a = 0.1 # our choice of learning rate
lamb = 0.0002 # our choice of lambda

In [195]:

# hold out some data to test the accuracy
held_x = x[0:100]
held_y = y[0:100]
x = x[100:]
y = y[100:]

print(len(held_x), len(held_y))
print(len(x), len(y))

100 100
2900 2900


In [215]:
len(x)

3000

In [226]:
# try running SGD here
weight = SGD(x, y, w, a, lamb)

print(weight)

2079.4415416797233 0.5 1
epoch: 1
511.1336707774801 0.04707746760443521 0
epoch: 2
240.5169019041291 0.2898746776143355 0
epoch: 3
155.0368171771063 0.908151272510554 1
epoch: 4
14.836873056600552 0.7501693028256219 1
epoch: 5
168.38410704956414 0.08564085114093514 0
epoch: 6
53.36729617588696 0.7547551648399894 1
epoch: 7
49.58841247568262 0.8796911174267457 1
epoch: 8
18.49798874079022 0.8607599294305872 1
epoch: 9
10.011444613871276 0.34385395266145147 0
epoch: 10
56.0693766466884 0.8467271556581496 1
epoch: 11
26.663990157714352 0.9741129938392393 1
epoch: 12
7.564855308977599 0.14255143312397703 0
epoch: 13
15.07630049106126 0.08482156102765068 0
epoch: 14
50.72738207978841 0.8719025625029153 1
epoch: 15
228.68358380096026 0.015366765363063146 0
epoch: 16
217.68214659441833 0.011366236478713935 0
epoch: 17
17.845273736318518 0.5078783178372178 1
epoch: 18
13.344816971928935 0.8987324932212856 1
epoch: 19
1.830563738549131 0.5086383738781161 1
epoch: 20
8.267546115583968 0.92039948

0.23062126780632752 0.8942505255597423 1
epoch: 165
0.20216371502147013 0.27844249796378423 0
epoch: 166
0.584103863196674 0.4038612502708753 1
epoch: 167
0.4265949433793139 0.27880518442676144 0
epoch: 168
0.0911212745679677 0.05728715562243437 0
epoch: 169
0.44355304263922335 0.7589531325254785 1
epoch: 170
0.4317147044137073 0.12637835698495292 0
epoch: 171
0.04825422563055781 0.4829457552080905 0
epoch: 172
0.027565995264581034 0.33450541828822866 0
epoch: 173
0.8629590405183762 0.18640807673167326 0
epoch: 174
0.8770983495122664 0.8126713402960335 1
epoch: 175
0.08245685694976146 0.3856124350923116 0
epoch: 176
0.057316024623560224 0.5734999314085114 1
epoch: 177
0.06202419352814559 0.710162725627711 1
epoch: 178
0.40305351212452933 0.17387414214149985 0
epoch: 179
0.4224512849702933 0.12688201616429856 0
epoch: 180
0.17870207132887117 0.390432603653045 0
epoch: 181
0.13595754030825447 0.2986861093785325 0
epoch: 182
0.19760476109968295 0.7779294895777163 1
epoch: 183
0.1924781679

In [227]:
predict(weight, test_x)

In [148]:
shuffled_x
shuffled_y

array([0, 1, 1, ..., 0, 1, 1])